In [1]:
import tkinter as tk
from tkinter import messagebox, ttk, Frame
from PIL import Image
import cv2
import numpy as np
import os
import mysql.connector

# Initialize Main Window
window = tk.Tk()
window.title("Advanced Face Recognition System")
window.geometry("800x600")
window.configure(bg='#2c2f33')

# Create a Notebook (Tabbed Interface)
notebook = ttk.Notebook(window)
notebook.pack(pady=10, expand=True)

# Create Frames for Tabs
home_frame = Frame(notebook, width=800, height=500, bg='#23272a')
train_frame = Frame(notebook, width=800, height=500, bg='#23272a')
detect_frame = Frame(notebook, width=800, height=500, bg='#23272a')

home_frame.pack(fill='both', expand=True)
train_frame.pack(fill='both', expand=True)
detect_frame.pack(fill='both', expand=True)

# Add Frames to Notebook
notebook.add(home_frame, text='Home')
notebook.add(train_frame, text='Train Classifier')
notebook.add(detect_frame, text='Detect Faces')

# Style Configuration
style = ttk.Style()
style.configure('TButton', font=('Helvetica', 14), padding=10)
style.configure('TLabel', background='#2c2f33', foreground='white', font=('Helvetica', 16))

# Variables for Input
name_var = tk.StringVar()
age_var = tk.StringVar()
address_var = tk.StringVar()

# Face Cascade for Detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# ================== Home Tab ==================
ttk.Label(home_frame, text="Enter User Details", font=('Helvetica', 20)).pack(pady=10)

form_frame = Frame(home_frame, bg='#2c2f33')
form_frame.pack(pady=10)

ttk.Label(form_frame, text="Name:").grid(row=0, column=0, padx=10, pady=10)
ttk.Entry(form_frame, textvariable=name_var, width=30).grid(row=0, column=1, padx=10, pady=10)

ttk.Label(form_frame, text="Age:").grid(row=1, column=0, padx=10, pady=10)
ttk.Entry(form_frame, textvariable=age_var, width=30).grid(row=1, column=1, padx=10, pady=10)

ttk.Label(form_frame, text="Address:").grid(row=2, column=0, padx=10, pady=10)
ttk.Entry(form_frame, textvariable=address_var, width=30).grid(row=2, column=1, padx=10, pady=10)

def generate_dataset():
    if name_var.get() == "" or age_var.get() == "" or address_var.get() == "":
        messagebox.showinfo('Error', 'All fields are required!')
        return

    # Database connection
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        passwd="",
        database="Authorized_user"
    )
    mycursor = mydb.cursor()

    # Get the highest ID and increment it
    mycursor.execute("SELECT MAX(id) FROM my_table")
    result = mycursor.fetchone()
    user_id = (result[0] or 0) + 1

    # Insert into database
    sql = "INSERT INTO my_table (id, name, age, address) VALUES (%s, %s, %s, %s)"
    val = (user_id, name_var.get(), age_var.get(), address_var.get())
    mycursor.execute(sql, val)
    mydb.commit()

    # Dataset Generation
    cap = cv2.VideoCapture(0)
    img_id = 0

    progress['value'] = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            img_id += 1
            face = gray[y:y + h, x:x + w]
            file_name = f"data/user.{user_id}.{img_id}.jpg"
            cv2.imwrite(file_name, face)
            cv2.imshow("Cropped Face", face)

        progress['value'] = (img_id / 100) * 100
        window.update_idletasks()

        if cv2.waitKey(1) == 13 or img_id == 100:
            break

    cap.release()
    cv2.destroyAllWindows()
    messagebox.showinfo('Result', 'Dataset Generation Completed')

# Progress Bar for Dataset Generation
progress = ttk.Progressbar(home_frame, orient='horizontal', length=400, mode='determinate')
progress.pack(pady=10)

ttk.Button(home_frame, text="Generate Dataset", command=generate_dataset).pack(pady=10)

# ================== Train Classifier Tab ==================
ttk.Label(train_frame, text="Train the Classifier", font=('Helvetica', 20)).pack(pady=10)

def train_classifier():
    data_dir = "data"
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]

    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')
        image_np = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])
        faces.append(image_np)
        ids.append(id)

    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, np.array(ids))
    clf.write("classifier.xml")
    messagebox.showinfo('Result', 'Training Completed')

ttk.Button(train_frame, text="Train Classifier", command=train_classifier).pack(pady=20)

# ================== Detect Faces Tab ==================
ttk.Label(detect_frame, text="Detect Faces", font=('Helvetica', 20)).pack(pady=10)

def get_name_from_db(user_id):
    """Retrieve the name of the user based on the ID from the database."""
    try:
        mydb = mysql.connector.connect(
            host="localhost",
            user="root",
            passwd="",
            database="Authorized_user"
        )
        mycursor = mydb.cursor()
        mycursor.execute(f"SELECT name FROM my_table WHERE id = {user_id}")
        result = mycursor.fetchone()
        return result[0] if result else "Unknown"
    except mysql.connector.Error as err:
        print(f"Database error: {err}")
        return "Unknown"

def detect_faces():
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")

    cap = cv2.VideoCapture(0)

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)

            for (x, y, w, h) in faces:
                id, confidence = clf.predict(gray[y:y + h, x:x + w])

                # Set a threshold for confidence level
                if confidence < 100:
                    name = get_name_from_db(id)
                else:
                    name = "Unknown"

                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                # Determine color based on recognition status
                color = (0, 0, 255) if name == "Unknown" else (0, 255, 0)
                cv2.putText(frame, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

            cv2.imshow("Face Detection", frame)

            if cv2.waitKey(1) & 0xFF == 27:  # ESC key to exit
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()

ttk.Button(detect_frame, text="Detect Faces", command=detect_faces).pack(pady=20)

# Run the Application
window.mainloop()
